In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.utils as vutils
from collections import Counter

# Define dataset path
dataset_path = "./images"

# Define transformations
transform = transforms.Compose([
    transforms.Resize((64, 64)),  
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  
])

# Load dataset
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Create DataLoader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

# Get class distribution
class_counts = Counter(dataset.targets)
max_count = max(class_counts.values())  # Find max count to balance
print("Class Distribution:", class_counts)

# Calculate images needed for each class
images_needed = {cls: max_count - count for cls, count in class_counts.items()}
print("Images Needed Per Class:", images_needed)

# Get class index mappings
idx_to_class = {v: k for k, v in dataset.class_to_idx.items()}  # Reverse mapping


c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


Class Distribution: Counter({5: 288, 6: 274, 8: 206, 2: 205, 0: 142, 7: 125, 1: 120, 3: 108, 4: 80})
Images Needed Per Class: {0: 146, 1: 168, 2: 83, 3: 180, 4: 208, 5: 0, 6: 14, 7: 163, 8: 82}


In [2]:
# class Generator(nn.Module):
#     def __init__(self, latent_dim=100):
#         super(Generator, self).__init__()
#         self.model = nn.Sequential(
#             nn.Linear(latent_dim, 256),
#             nn.ReLU(),
#             nn.Linear(256, 512),
#             nn.ReLU(),
#             nn.Linear(512, 1024),
#             nn.ReLU(),
#             nn.Linear(1024, 64 * 64 * 3),
#             nn.Tanh()
#         )

#     def forward(self, z):
#         img = self.model(z)
#         img = img.view(img.size(0), 3, 64, 64)
#         return img
    
# class Discriminator(nn.Module):
#     def __init__(self):
#         super(Discriminator, self).__init__()
#         self.model = nn.Sequential(
#             nn.Linear(64 * 64 * 3, 512),
#             nn.ReLU(),
#             nn.Linear(512, 256),
#             nn.ReLU(),
#             nn.Linear(256, 1),
#             nn.Sigmoid()
#         )

#     def forward(self, img):
#         img = img.view(img.size(0), -1)  
#         return self.model(img)


In [3]:
# # Set device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Initialize models
# latent_dim = 100
# generator = Generator(latent_dim).to(device)
# discriminator = Discriminator().to(device)

# # Optimizers & Loss
# optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
# optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
# criterion = nn.BCELoss()

# # Train GAN
# num_epochs = 5000
# batch_size = 64

# for epoch in range(num_epochs):
#     z = torch.randn(batch_size, latent_dim).to(device)
#     fake_imgs = generator(z)

#     # Train Discriminator
#     real_imgs, _ = next(iter(dataloader))
#     real_imgs = real_imgs.to(device)

#     real_validity = discriminator(real_imgs)
#     fake_validity = discriminator(fake_imgs.detach())

#     real_loss = criterion(real_validity, torch.ones_like(real_validity))
#     fake_loss = criterion(fake_validity, torch.zeros_like(fake_validity))
#     d_loss = (real_loss + fake_loss) / 2

#     optimizer_D.zero_grad()
#     d_loss.backward()
#     optimizer_D.step()

#     # Train Generator
#     fake_validity = discriminator(fake_imgs)
#     g_loss = criterion(fake_validity, torch.ones_like(fake_validity))

#     optimizer_G.zero_grad()
#     g_loss.backward()
#     optimizer_G.step()

#     if epoch % 500 == 0:
#         print(f"Epoch {epoch}: D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import os
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder

# Hyperparameters
latent_dim = 100
batch_size = 64
image_size = 64
epochs = 5000
lr_g = 0.0002  # Lowered generator learning rate
lr_d = 0.0001  # Lowered discriminator learning rate
beta1 = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Transformations
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load dataset
dataset_path = "./images"
dataset = ImageFolder(root=dataset_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(True),
            nn.BatchNorm1d(256),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.BatchNorm1d(512),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.BatchNorm1d(1024),
            nn.Linear(1024, image_size * image_size * 3),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.main(z)
        img = img.view(-1, 3, image_size, image_size)
        return img

# Define Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(image_size * image_size * 3, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.main(img_flat)
        return validity




cuda


In [3]:
# Initialize models
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Loss and Optimizers
criterion = nn.BCELoss()
optimizer_g = optim.Adam(generator.parameters(), lr=lr_g, betas=(beta1, 0.999))
optimizer_d = optim.Adam(discriminator.parameters(), lr=lr_d, betas=(beta1, 0.999))

# Training Loop
for epoch in range(epochs):
    for i, (real_imgs, _) in enumerate(dataloader):

        # Move data to device
        real_imgs = real_imgs.to(device)
        batch_size = real_imgs.shape[0]

        # Generate noise
        noise = torch.randn(batch_size, latent_dim, device=device)
        fake_imgs = generator(noise)

        # Create labels with smoothing
        real_labels = torch.full((batch_size, 1), 0.9, device=device)  # Use 0.9 instead of 1
        fake_labels = torch.full((batch_size, 1), 0.1, device=device)  # Use 0.1 instead of 0

        # Train Discriminator
        optimizer_d.zero_grad()
        real_loss = criterion(discriminator(real_imgs), real_labels)
        fake_loss = criterion(discriminator(fake_imgs.detach()), fake_labels)
        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_d.step()

        # Train Generator twice as often
        if i % 2 == 0:
            optimizer_g.zero_grad()
            g_loss = criterion(discriminator(fake_imgs), real_labels)
            g_loss.backward()
            optimizer_g.step()

    # Print loss and save generated images
    if epoch % 500== 0:
        print(f"Epoch [{epoch}/{epochs}] | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f}")
        vutils.save_image(fake_imgs.data[:25], f"generated_{epoch}.png", normalize=True)

print("Training complete!")

Epoch [0/5000] | D Loss: 0.7576 | G Loss: 1.7919
Epoch [500/5000] | D Loss: 0.8012 | G Loss: 1.6630
Epoch [1000/5000] | D Loss: 0.6997 | G Loss: 2.0467
Epoch [1500/5000] | D Loss: 0.7047 | G Loss: 2.1497
Epoch [2000/5000] | D Loss: 0.6535 | G Loss: 2.0900
Epoch [2500/5000] | D Loss: 0.6543 | G Loss: 2.0601
Epoch [3000/5000] | D Loss: 0.6521 | G Loss: 2.0053
Epoch [3500/5000] | D Loss: 0.6529 | G Loss: 2.0612
Epoch [4000/5000] | D Loss: 0.7868 | G Loss: 1.8345
Epoch [4500/5000] | D Loss: 0.8148 | G Loss: 1.6966
Training complete!


In [ ]:
# Generate images per class
for class_label, num_images in images_needed.items():
    if num_images <= 0:
        continue  

    save_path = os.path.join(dataset_path, idx_to_class[class_label])  
    os.makedirs(save_path, exist_ok=True)  

    z = torch.randn(num_images, latent_dim).to(device)
    synthetic_imgs = generator(z) 

    for i, img in enumerate(synthetic_imgs):
        vutils.save_image(img, f"{save_path}/{idx_to_class[class_label]}_fake_{i}.png")

    print(f"✅ {num_images} synthetic images saved for class '{idx_to_class[class_label]}'")


✅ 146 synthetic images saved for class 'chatamari'
✅ 168 synthetic images saved for class 'chhoila'
✅ 83 synthetic images saved for class 'dalbhat'
✅ 180 synthetic images saved for class 'dhindo'
✅ 208 synthetic images saved for class 'gundruk'
✅ 14 synthetic images saved for class 'momo'
✅ 163 synthetic images saved for class 'sekuwa'
✅ 83 synthetic images saved for class 'selroti'


In [ ]:
import collections

# Count final images
final_counts = collections.Counter()
for class_label in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_label)
    if os.path.isdir(class_dir):
        final_counts[class_label] = len(os.listdir(class_dir))

print("📊 Final Balanced Class Distribution:", final_counts)


📊 Final Balanced Class Distribution: Counter({'chatamari': 288, 'chhoila': 288, 'dalbhat': 288, 'dhindo': 288, 'gundruk': 288, 'kheer': 288, 'momo': 288, 'sekuwa': 288, 'selroti': 288})
